# Lab 9 - Detección de Anomalías con Autoencoder, Isolation Forest y LOF

In [ ]:
#!pip install -r requirements

In [1]:
from sklearn.datasets import fetch_covtype
import numpy as np 
import pandas as pd


In [2]:
bunch = fetch_covtype(as_frame=True)

df: pd.DataFrame = bunch.data 
srs: pd.Series = bunch.target

print(df.shape, srs.shape)

(581012, 54) (581012,)


In [3]:
type(df)

pandas.core.frame.DataFrame

In [9]:
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2590.0,56.0,2.0,212.0,-6.0,390.0,220.0,235.0,151.0,6225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2804.0,139.0,9.0,268.0,65.0,3180.0,234.0,238.0,135.0,6121.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2785.0,155.0,18.0,242.0,118.0,3090.0,238.0,238.0,122.0,6211.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2595.0,45.0,2.0,153.0,-1.0,391.0,220.0,234.0,150.0,6172.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
train_raw, test_raw = train_test_split(df, test_size=0.2, random_state=32)
train_raw, val_raw  = train_test_split(train_raw, test_size=0.25, random_state=32)

### Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

# scale all columns except binary ones
def scale_nonbin(df: pd.DataFrame) -> pd.DataFrame:
     binary_cols = [col for col in df.columns 
                   if set(df[col].dropna().unique()).issubset({0, 1})]
     cols_to_scale = [col for col in df.columns if col not in binary_cols]
     scaler = StandardScaler()
     df_scaled = df.copy()
     df_scaled[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
     return df_scaled
     


In [8]:
# standardize all except binary columns
df_scaled : pd.DataFrame = scale_nonbin(df=df)
df_scaled.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,-1.297805,-0.935157,-1.482820,-0.053767,-0.796273,-1.180146,0.330743,0.439143,0.142960,3.246283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.319235,-0.890480,-1.616363,-0.270188,-0.899197,-1.257106,0.293388,0.590899,0.221342,3.205504,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.554907,-0.148836,-0.681563,-0.006719,0.318742,0.532212,0.816364,0.742654,-0.196691,3.126965,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.622768,-0.005869,0.520322,-0.129044,1.227908,0.474492,0.965786,0.742654,-0.536343,3.194931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.301377,-0.988770,-1.616363,-0.547771,-0.813427,-1.256464,0.293388,0.540313,0.195215,3.165479,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Divive into Train/Val/Test

In [12]:
# find abnormal data
def filter_abnormal_data(df: pd.DataFrame, threshold: float = 3.5) -> pd.DataFrame:
    med = df.median()
    mad = (df - med).abs().median()  # median absolute deviation (L1-ish)
    # avoid div-by-zero
    mad = mad.replace(0, 1e-12)
    z = (df - med) / mad
    mask: pd.DataFrame = (z.abs() < threshold).all(axis=1)# type: ignore[return-value]
    return df[mask]

La data se filtra y se excluye a los valores atípicos porque el modelo de autoencoder se entrena con valores meramente normales. Para así poder encontrar anomalías. Si en caso dejáramos valores normales junto con atípicos, la capacidad de encontrar valores anormales del modelo sería peor porque no podría distinguirlos ya que fue entrenado con ambos. 